<a href="https://colab.research.google.com/github/karthik1946/01_c/blob/main/dl_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd

# Set display options to show more columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

file_names = ["/content/drive/MyDrive/S-1/Train/1.txt", "/content/drive/MyDrive/S-1/Train/2.txt", "/content/drive/MyDrive/S-1/Train/3.txt", "/content/drive/MyDrive/S-1/Train/4.txt", "/content/drive/MyDrive/S-1/Train/5.txt", "/content/drive/MyDrive/S-1/Train/6.txt", "/content/drive/MyDrive/S-1/Train/7.txt", "/content/drive/MyDrive/S-1/Train/8.txt"]

concatenated_data = pd.DataFrame()

for file_name in file_names:
    data = pd.read_csv(file_name, delimiter=',', header=None)
    concatenated_data = pd.concat([concatenated_data, data], axis=0)

concatenated_data.reset_index(drop=True, inplace=True)

concatenated_data.shape


(194369, 38)

In [ ]:
pip install tensorflow

In [ ]:
import tensorflow

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
import pandas as pd

# Set display options to show more columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

file_names = ["/content/drive/MyDrive/S-1/Test_Label/1.txt",
              "/content/drive/MyDrive/S-1/Test_Label/2.txt",
              "/content/drive/MyDrive/S-1/Test_Label/3.txt",
              "/content/drive/MyDrive/S-1/Test_Label/4.txt",
              "/content/drive/MyDrive/S-1/Test_Label/5.txt",
              "/content/drive/MyDrive/S-1/Test_Label/6.txt",
              "/content/drive/MyDrive/S-1/Test_Label/7.txt"]

X_test = pd.DataFrame()

for file_name in file_names:
    data = pd.read_csv(file_name, delimiter=',', header=None)
    X_test = pd.concat([X_test, data], axis=0)

X_test.reset_index(drop=True, inplace=True)

print("Shape of predictions:", X_test.shape)



Shape of predictions: (170675, 1)


In [ ]:
import pandas as pd

# Set display options to show more columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

file_names = ["/content/drive/MyDrive/S-1/Test/1.txt", "/content/drive/MyDrive/S-1/Test/2.txt", "/content/drive/MyDrive/S-1/Test/3.txt", "/content/drive/MyDrive/S-1/Test/4.txt", "/content/drive/MyDrive/S-1/Test/5.txt", "/content/drive/MyDrive/S-1/Test/6.txt", "/content/drive/MyDrive/S-1/Test/7.txt"]

X_data = pd.DataFrame()

for file_name in file_names:
    data = pd.read_csv(file_name, delimiter=',', header=None)
    X_data = pd.concat([X_data, data], axis=0)

X_data.reset_index(drop=True, inplace=True)

X_data.shape


(170675, 38)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming concatenated_data is your DataFrame with 38 sensor columns
multivariate_data = concatenated_data.copy()

# Standardize the data
scaler = StandardScaler()
scaled_data = scaler.fit_transform(multivariate_data)

# Define sequences for training
def create_sequences(data, sequence_length):
    sequences = []
    for i in range(len(data) - sequence_length + 1):
        seq = data[i:i + sequence_length, :]
        sequences.append(seq)
    return np.array(sequences)

sequence_length = 10  # Adjust based on your requirements
X = create_sequences(scaled_data, sequence_length)

# Split the data into training and validation sets
X_train, X_val = train_test_split(X, test_size=0.2, shuffle=False)

# Build the LSTM model
model = Sequential()
model.add(LSTM(units=50, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(units=X_train.shape[2]))  # Output layer with the same number of features as sensors
model.compile(optimizer='adam', loss='mean_squared_error')

# Define a ModelCheckpoint callback to save the best weights
checkpoint = ModelCheckpoint("best_model_weights.h5", save_best_only=True, save_weights_only=True, monitor='val_loss', mode='min', verbose=1)

# Train the model
epochs = 3  # Adjust the number of epochs based on your requirements
best_val_accuracy = 0.0
best_threshold = 0.4551  # Initial threshold
for epoch in range(epochs):
    history = model.fit(X_train, X_train[:, -1, :], epochs=1, batch_size=32, validation_data=(X_val, X_val[:, -1, :]), callbacks=[checkpoint], verbose=1)

    # Calculate validation accuracy after each epoch
    predictions_val = model.predict(X_val)
    mse_val = np.mean(np.square(X_val[:, -1, :] - predictions_val), axis=1)

    # Calculate dynamic threshold based on the validation MSE
    threshold_val = np.percentile(mse_val, 95)  # Example: Use the 95th percentile of validation MSE as the threshold

    anomalies_val = (mse_val > threshold_val).astype(int)
    zeros_list = np.zeros_like(anomalies_val)
    validation_accuracy = np.sum(anomalies_val == zeros_list) / zeros_list.shape[0]

    # Adjust threshold dynamically based on validation accuracy
    if validation_accuracy > best_val_accuracy:
        best_val_accuracy = validation_accuracy
        best_threshold = threshold_val

    print(f"Epoch {epoch + 1}/{epochs} - Validation Accuracy: {validation_accuracy * 100:.2f}% - Threshold: {best_threshold:.4f}")


4858/4859 [============================>.] - ETA: 0s - loss: 0.1546
Epoch 1: val_loss improved from inf to 0.25830, saving model to best_model_weights.h5
1215/1215 [==============================] - 5s 4ms/step
Epoch 1/3 - Validation Accuracy: 95.00% - Threshold: 0.5875
4858/4859 [============================>.] - ETA: 0s - loss: 0.1111
Epoch 1: val_loss improved from 0.25830 to 0.22208, saving model to best_model_weights.h5
1215/1215 [==============================] - 5s 4ms/step
Epoch 2/3 - Validation Accuracy: 95.00% - Threshold: 0.5875
4856/4859 [============================>.] - ETA: 0s - loss: 0.1044
Epoch 1: val_loss improved from 0.22208 to 0.20430, saving model to best_model_weights.h5
1215/1215 [==============================] - 4s 3ms/step
Epoch 3/3 - Validation Accuracy: 95.00% - Threshold: 0.5875


In [ ]:
# Assuming you have a DataFrame named X_test with 38 sensor columns
X_test_np = X_test.to_numpy()

# Create sequences for testing
X_test_sequences = create_sequences(X_test_np, sequence_length)

# Reshape X_test_sequences to match the input shape of the model
X_test_sequences = X_test_sequences.reshape(-1, sequence_length, X_test_np.shape[1])

# Use the trained model to predict sequences on the test data
predictions_test = model.predict(X_test_sequences)

# Reshape predictions to match the shape of X_test
predictions_test = predictions_test[:, 0, :].reshape(-1, X_test_np.shape[1])

# Now, you can use predictions_test for further analysis or evaluation


ValueError: ignored